In [1]:
%matplotlib inline
import xarray as xr
import numpy as np
import datetime as dt
from datetime import timedelta
from opendrift.readers import reader_netCDF_CF_generic
from opendrift.readers import reader_global_landmask
from opendrift.models.oceandrift import OceanDrift
import os,glob
from tqdm import tqdm

import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

from matplotlib import cm
import pandas as pd

from IPython.display import HTML, display
from mpl_toolkits.axes_grid1 import make_axes_locatable

import seaborn as sns

In [7]:
mask_gg = [(37.78008, 37.83338),(-122.50854, -122.47301)]

In [2]:
def make_map():
    fig = plt.figure(figsize=(16,8))
    cart_proj = ccrs.PlateCarree()
    ax = plt.axes(projection=cart_proj)
    ax.coastlines('10m', linewidth=0.8,zorder=200)
    ax.add_feature(cfeature.LAND)
    ax.add_feature(cfeature.OCEAN)
    ax.set_xlim(-123,-122.26)
    ax.set_ylim(37.5,38.1)
    
    return(fig, ax)

## Plotting the data ##

__1. Old model runs are found under Data/model_outputs/sf_bay_start_time__
First, lets start by plotting a model run

In [70]:
for fname in tqdm(sorted(glob.glob("../DailyModelRuns/model_output/*_continuous.nc"))[:]):
    SAVE_NAME = os.path.basename(fname).split("_")[2] + "_continuous.png"
    SAVE_DIR = "/home/pdaniel/SurfaceCurrentMaps/PlumeTrackerNotebooks/Data/model_output/static/"
    start_time_dt = pd.to_datetime(os.path.basename(fname).split("_")[2])

    ds = xr.open_dataset(fname)
    df = ds['status'].to_dataframe()
    df['start_time'] = 0
    experimentStart = df.iloc[0].name[1]
    df_flat = df.reset_index()
    df_flat = df_flat.query("status >= 0")
    first_mapping = df_flat.groupby(by=['trajectory'])['time'].first()
    first_hours = first_mapping - experimentStart
    first_hours = first_hours.apply(lambda x: x.seconds/3600)
    df_flat['start_hour'] = df_flat['trajectory'].map(first_hours)
    
    fig, ax = make_map()
    sns.set_theme(style="darkgrid")
    sns.set_context('talk')
    normalize = mcolors.Normalize(vmin=0, vmax=2)
    for i in range(1, df_flat.trajectory.max()+1):
        new_df = df_flat.query("trajectory == @i")
        if i == 1:
            sLat = [new_df.iloc[0].lat]
            sLon = [new_df.iloc[0].lon]
            ax.scatter(sLon, sLat, zorder=220, c='r', s=70, label='Seed Start')
        if (i % 2) == 0:

            eLat = [new_df.iloc[-1].lat][0]
            eLon = [new_df.iloc[-1].lon][0]
            if not ((eLat > mask_gg[0][0]) & (eLat < mask_gg[0][1]) & (eLon > mask_gg[1][0]) & (eLon < mask_gg[1][1])):
                
                start_hours = new_df.iloc[0].start_hour 
                if start_hours > 3:
                    break
                cax = ax.scatter(eLon, eLat,
                                 zorder=200,
                                 c=start_hours,
                                 norm=normalize,
                                 marker='X',
                                 edgecolor='.25',
                                 cmap='binary',
                                 s=150)
                ax.plot(new_df['lon'],new_df['lat'],color='.5',lw=.5)
    plt.colorbar(cax,label='Released Time [Hours]')
    plt.text(0,-0.03,"Release Time: "+str(start_time_dt),
        horizontalalignment='left',
        verticalalignment='center', 
        transform=ax.transAxes)
    plt.text(0,-0.071,"End Time: "+str(start_time_dt + dt.timedelta(hours=48)),
        horizontalalignment='left',
        verticalalignment='center', 
        transform=ax.transAxes)
    plt.text(0,1.02,"SF Outflow Tracker, 48 Hour Trajectories",
        horizontalalignment='left',
        verticalalignment='center', 
        fontweight='bold',
        transform=ax.transAxes)

    plt.text(0,-.115,"Particles are released every 15 minutes for the first two hours and forced for 48 hours using 2km surface current maps measured by HFR.",
        horizontalalignment='left',
        verticalalignment='center', 
        fontsize=10,
        transform=ax.transAxes)


    
    colors = ['red']
    labels = ['Start']
    plt.legend(
        bbox_to_anchor=(0.87,-0.08,.1,.1),
        markerfirst=False,
        frameon=False,
        fancybox=False
              )
    
    plt.show()

    plt.savefig(fname=os.path.join(SAVE_DIR,SAVE_NAME), dpi=200, bbox_inches='tight', pad_inches=0.15)
    plt.close()


100%|██████████| 270/270 [09:36<00:00,  2.14s/it]


In [32]:
os.path.basename(fname).split("_")[2]

'20210120T142400'

In [7]:
t.name[1]

NameError: name 't' is not defined

In [9]:
time_steps = df.index.get_level_values(1).values
t = time_steps[48]
dt = time_steps[24]
step = df.query("time <= @t & time >= @dt & status == 0")

In [ ]:
time_steps

In [11]:
time_steps = df.index.get_level_values(1).values
norm = mcolors.Normalize(vmin=0, vmax=25)
for i in range(time_steps.shape[0]):
    fig, ax = make_map()
    t = time_steps[0]
    if i < 24:
        dt = time_steps[0]
    else:
        dt = time_steps[i -  24]
    step = df.query("time <= @t & time >= @dt & status == 0")
    for traj, temp_df in step.groupby(level=0):
        ax.plot(temp_df['lon'], temp_df['lat'],color='.75')
        mod = traj//100
        marker = temp_df.iloc[-1]
        ax.scatter(marker['lon'],marker['lat'],c=norm(mod),zorder=10)
        plt.savefig(f"Data/model_output/animation_cont/sfBay_cont_01_frame_{i:04}.png")
        plt.close()

  fig = plt.figure(figsize=(12,9))



KeyboardInterrupt: 

Error in callback <function install_repl_displayhook.<locals>.post_execute at 0x7f6ab07588b0> (for post_execute):


KeyboardInterrupt: 

In [11]:
max_val = surface_map['density_surface'].max()
fig, ax = make_map()
current_step = surface_map.isel(time=2)
im = current_step['density_surface'].plot(ax=ax, vmin=0, vmax=max_val, add_colorbar=False)
plt.colorbar(im, fraction=0.026, pad=0.02,)
plt.show()

In [12]:
max_val = surface_map['density_surface'].max()

for i in range(surface_map['time'].shape[0]):
    fig, ax = make_map()
    current_step = surface_map.isel(time=i)
    im = current_step['density_surface'].plot(ax=ax, vmin=0, vmax=max_val, add_colorbar=False)
    plt.colorbar(im, fraction=0.026, pad=0.02)
    plt.text(.02,.95,"time_step: {}".format(i), transform=ax.transAxes)
    plt.savefig(f"Data/model_output/animation/sfBay_density_large_01_frame_{i:04}.png")
    plt.close()

In [13]:
!magick convert Data/model_output/animation/sfBay_density_large_01_frame*png Data/model_output/density_animation_large.gif
display(HTML("<img src='Data/model_output/density_animation_large.gif' />"))

/bin/bash: magick: command not found


In [3]:
import glob
from PIL import Image

In [5]:
# filepaths
fp_in = "Data/model_output/animation/sfBay_density_large_01_frame*png"
fp_out = "Data/model_output/animation/density_animation_large.gif"

# https://pillow.readthedocs.io/en/stable/handbook/image-file-formats.html#gif
img, *imgs = [Image.open(f) for f in sorted(glob.glob(fp_in))]
img.save(fp=fp_out, format='GIF', append_images=imgs,
         save_all=True, duration=200, loop=0)


NameError: name 'o' is not defined